<a href="https://colab.research.google.com/github/alangan17/smazc01/blob/main/SMAZC_01_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Markets Analytics Zoomcamp Week1

Repo: https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/tree/main/01-intro-and-data-sources

# Dependencies

In [3]:
!pip install yfinance

In [4]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# Introduction


In this homework, we're going to download finance data from various sources and make simple calculations/analysis.


# Question 1

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1).
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

In [10]:
start = date(year=2022, month=1, day=1)

In [45]:
df = pdr.DataReader("GDPC1", "fred", start=start)
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    8 non-null      datetime64[ns]
 1   GDPC1   8 non-null      float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 256.0 bytes


In [46]:
df['gdp_growth_yoy'] = df.GDPC1/df.GDPC1.shift(4)-1
df.tail(8)

,DATE,GDPC1,gdp_growth_yoy
0,2022-01-01,21738.871,NaN
1,2022-04-01,21708.160,NaN
2,2022-07-01,21851.134,NaN
3,2022-10-01,21989.981,NaN
4,2023-01-01,22112.329,0.017179
5,2023-04-01,22225.350,0.023825
6,2023-07-01,22490.692,0.029269
7,2023-10-01,22679.255,0.031345


In [59]:
df_2023 = df[df['DATE'].dt.year == 2023]
average_gdpc1_us_yoy_2023 = round(df_2023['gdp_growth_yoy'].mean() * 100, 1)
print(f"Average Year to Year GDP Growth in 2023: {average_gdpc1_us_yoy_2023}%")

Average Year to Year GDP Growth in 2023: 2.5%
